# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [2]:
data = load_wine()
X = pd.DataFrame(data['data'])[[9, 11, 12]].values
y = data.target

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=17)

# x_train, x_test, y_train, y_test



**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [3]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
skf



StratifiedKFold(n_splits=5, random_state=17, shuffle=True)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [19]:
lr = LogisticRegression(random_state=17, multi_class='ovr')
lr.fit(x_train, y_train)
scores = cross_val_score(lr, X, y, cv=skf)
print(f'Средняя доля правильных ответов при кросс-валидации: {scores.mean()}')
y_pred_lr = lr.predict(x_test)
acc = accuracy_score(y_test, y_pred_lr)
print(f'Доля правильных ответов на тестовой выборке: {acc}')



Средняя доля правильных ответов при кросс-валидации: 0.9330158730158731
Доля правильных ответов на тестовой выборке: 0.9111111111111111


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [7]:
from sklearn.svm import SVC

In [27]:
svc = SVC(random_state=17, decision_function_shape='ovr')
svc.fit(x_train, y_train)
scores = cross_val_score(svc, X, y, cv=skf)
print(f'Средняя доля правильных ответов при кросс-валидации: {scores.mean()}')
y_pred_svc = svc.predict(x_test)
acc = accuracy_score(y_test, y_pred_svc)
print(f'Доля правильных ответов на тестовой выборке: {acc}')







Средняя доля правильных ответов при кросс-валидации: 0.6803174603174603
Доля правильных ответов на тестовой выборке: 0.6222222222222222


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [22]:
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
cm_lr = confusion_matrix(y_pred_lr, y_test)
cm_svc = confusion_matrix(y_pred_svc, y_test)

print(cm_lr) # '4' -- 1 и 2 класс
print()
print(cm_svc) # '15' -- 1 и 2 класс





[[ 9  0  0]
 [ 0 19  4]
 [ 0  0 13]]

[[ 9  0  2]
 [ 0 19 15]
 [ 0  0  0]]


### 5.
Для каждой модели выведите classification report.

In [33]:
print(classification_report(y_test, y_pred_lr))







              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45



In [34]:
print(classification_report(y_test, y_pred_svc))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



C:\Users\vadim.shepelev\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vadim.shepelev\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vadim.shepelev\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif